In [1]:
%matplotlib inline
import numpy as np

import jupyter_manim
from manim import *

Manim Community v0.13.1

In [4]:
%%manim -qm -v WARNING Teleportation

class Teleportation(Scene):
    config= {
        "x_lim":int,    # define leftmost limit of x for circuit and gates
        "y_lim":int     # define up-most limit of y for circuit and gates
    }
    def __init__(self, **kwargs):
        super(Teleportation, self).__init__()       #inheritance from class Scene
        self.x_lim=-5.5
        self.y_lim=2
    def construct(self):
        # List to hold all circuit line elements and VGroup  to include all the Mobjects of circuitline
        circuitline=[[None for i in range(5)] for i in range(5)]
        circuit=VGroup() # Vgroup 
        # Algorithm name
        algorithm="Teleportation Algorithm"
        title = Title(algorithm,include_underline=False).set_y(3.1)     #title of the ALgorithm
        self.add_foreground_mobject(title)

        #section zero: create all registers: quantum and classical
        regs=[MathTex(r"q_0"),MathTex(r"q_1"), MathTex(r"q_2"),MathTex(r"c_{rz}"), MathTex(r"c_{rxz}")] # list of registers name
        registers=VGroup()
        
        for i,reg in enumerate(regs):       #add registers to the scene
            reg.set_x(self.x_lim).set_y(2-i)
            circuitline[0][i]=Line([self.x_lim-0.5,2-i,0], [self.x_lim-0.6,2-i,0])
            registers.add(reg)
            circuit.add(circuitline[0][i])
        self.add(registers)
        dots=[None for i in range(5)]       # Dots 
        for i in range(3):
            dots[i]=Dot()

        
        def create_gate(gate:str,x:int,y:int,control=None,scale=1,textcolor=WHITE,boxcolor=BLUE):
            if control:
                dot=Dot().set_x(x).set_y(control)
                control_line= Line([x,control,0], [x,y,0])
                gate_ = MathTex(gate,color=textcolor).scale(scale+0.5).set_x(x).set_y(y)
                gate_.bg = BackgroundRectangle(gate_,stroke_width=1,color=boxcolor, fill_opacity=1).scale(scale+1)
                return VGroup(control_line,gate_.bg,gate_,dot)
            else:
                gate_ = MathTex(gate,color=textcolor).scale(scale+0.5).set_x(x).set_y(y)
                gate_.bg = BackgroundRectangle(gate_,stroke_width=1,color=boxcolor, fill_opacity=1).scale(scale+1)
                return VGroup(gate_.bg,gate_)
        
        def draw_circuitline(regs,sect:int,xrange:list,ydiff: float,barrier=True,circuitline=circuitline,circuit=circuit):
            for i in range(5):
                circuitline[sect][i]=Line([xrange[0],ydiff-i,-0.5], [xrange[1],ydiff-i,-0.5])
                circuit.add(circuitline[sect][i])
                self.add(circuitline[sect][i])
            if barrier:
                barrierline=DashedLine([xrange[1],2.2,0], [xrange[1],-0.25,0],dashed_ratio=0.75).set_opacity(0.5)
                self.add(barrierline)
                # self.add(VGroup(circuit,barrierline))
                # return VGroup(circuit,barrier)
        def move_dot(sect: int,dots=dots,circuitline=circuitline,circuit=circuit):
            animation=[]
            for j in range(3):
                animation.append(MoveAlongPath(dots[j], circuitline[sect][j]))
            # self.play(*animation)
            return animation
    

        # #section I: Create a maximally entangled state
        sect_range1=[self.x_lim+0.6,self.x_lim+3]
        draw_circuitline(regs,sect=1,xrange=sect_range1,ydiff=2)
        H_gate1=create_gate("H",self.x_lim+1.25 ,1,scale=0.5)
        C_not1= create_gate("X",self.x_lim+2.25 ,0,control=1,scale=0.5)

        texTemplate = TexTemplate()
        texTemplate.add_to_preamble(r"\usepackage{braket}")

        state_1a=MathTex(r"\ket{q_1 q_2}=\ket{0}",tex_template=texTemplate, font_size=20).set_x(sect_range1[0]).set_y(1.5)
        state_1b=MathTex(r"\ket{q_1 q_2}=\frac{1}{\sqrt{2}} (\ket{00} + \ket{11})",
                            tex_template=texTemplate, font_size=20).set_x(sect_range1[1]).set_y(1.5)
        state_psi=MathTex(r"\ket{q_0}=\alpha \ket{0} +\beta \ket{1}",
                            tex_template=texTemplate, font_size=20).set_x(sect_range1[0]).set_y(2.5)
        
        
        self.play(Create(H_gate1),Create(C_not1),Create(state_psi),Create(state_1a))
        self.play(state_psi.animate.move_to([sect_range1[1],2.5,0]),
                    FadeTransform(state_1a,state_1b),
                    *move_dot(sect=1),
                    runtime=3)

        # # Section :II create entanglement between 1st and 2nd qubit
        sect_range2=[self.x_lim+3,self.x_lim+5.5]
        draw_circuitline(regs,sect=2,xrange=sect_range2,ydiff=2)
        
        C_not2= create_gate("X",self.x_lim+3.75 ,1,control=2,scale=0.5)
        H_gate2=create_gate("H",self.x_lim+4.75 ,2,scale=0.5)

        # state_1=MathTex(r"\frac{1}{\sqrt{2}} (\ket{00} + \ket{11})",
        #                     tex_template=texTemplate, font_size=25).set_x(sect_range2[0]).set_y(1.5)
        state_2= MathTex(r"\ket{q_0 q_1 q_2} =\frac{1}{\sqrt{2}}(\ket{000} + \ket{011}+\ket{100} +\ket{111})",
                            tex_template=texTemplate, font_size=20).set_x(sect_range2[1]).set_y(2.5)
        
        self.play(Create(C_not2),Create(H_gate2),FadeOut(state_psi))
        self.play(FadeTransform(state_1b,state_2),
                    *move_dot(sect=2),
                    runtime=3)
        

        # # Section III: Add measurement Gates
        sect_range3=[self.x_lim+5.5,self.x_lim+8]
        draw_circuitline(regs,sect=3,xrange=sect_range3,ydiff=2)

        measure_0=create_gate(r"\not \frown",self.x_lim+6.25,y=-1,control=2,scale=0.5)
        measure_1=create_gate(r"\not \frown",self.x_lim+7.25,y=-2,control=1,scale=0.5)
        state_3= MathTex(r"\ket{q_0 q_1 q_2}= \frac{1}{2}( \ket{00}(\alpha \ket{0} + \beta \ket{1})\\ + \ket{01}( \alpha \ket{1} + \beta \ket{0})\\ +\ket{10} (\alpha \ket{0} - \beta \ket{1})\\ +\ket{11}(\alpha \ket{1} - \beta \ket{0}))",
                            tex_template=texTemplate, font_size=20).set_x(sect_range3[1]).set_y(1)

        self.play(Create(measure_0),Create(measure_1),FadeOut(state_2))
        self.play(FadeTransform(state_2,state_3),
                    dots[0].animate.move_to([self.x_lim+6.25,2,0]),
                    dots[1].animate.move_to([self.x_lim+7.25,1,0]),
                    FadeOut(dots[3]),
                    runtime=3)
        self.wait(2) 
        
        # Section IV: Add classical controlled gate
        sect_range4=[self.x_lim+8,self.x_lim+11]
        draw_circuitline(regs,sect=4,xrange=sect_range4,ydiff=2,barrier=False)
        classicX=create_gate(r"X",self.x_lim+8.75,y=0,control=-2,scale=0.5)
        classicZ=create_gate(r"Z",self.x_lim+9.75,y=0,control=-1,scale=0.5)

        state_4= MathTex(r"\ket{q_2}= \alpha \ket{0} +\beta \ket{1}",
                            tex_template=texTemplate, font_size=20).set_x(sect_range4[1]).set_y(0.4)

        self.play(Create(classicX),Create(classicZ),FadeOut(state_3),FadeTransform(state_3,state_4))  
        self.wait(3)     
        



In [1]:
import manim as manim
manim.__version__

Manim Community v0.13.1

'0.13.1'